* The Unsloth template for this notebook - https://colab.research.google.com/drive/1lN6hPQveB_mHSnTOYifygFcrO8C1bxq4?usp=sharing#scrollTo=Edrn7Rxmojtu

* The Phi-3.5.-mini-instruct model - https://huggingface.co/unsloth/Phi-3.5-mini-instruct

In [1]:
%%capture
!pip install unsloth "xformers==0.0.28.post2" --quiet
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" --quiet
!pip install datasets mlflow --quiet

In [2]:
from datetime import datetime
import mlflow
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import get_chat_template

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3.5-mini-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    #token = "", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.11.10: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 1994,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.11.10 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [21]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    )

In [22]:
def formatting_prompts_func(examples):
    convos = examples["task"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

In [29]:
from datasets import load_dataset
dataset = load_dataset('winterForestStump/cbr_bonds_info_detector')

In [30]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [31]:
print(dataset['train'][0]['text'])

<|user|>
#Instruction: You have to extract securities registration numbers from the provided text. #Provided text: Банк России 16.11.2023 принял решение о государственной регистрации изменений в программу структурных неконвертируемых процентных бездокументарных облигаций с залоговым обеспечением серии А-01, предназначенных для квалифицированных инвесторов, регистрационный номер программы 6-00669-R-001P, и изменений в решение о выпуске структурных неконвертируемых процентных бездокументарных облигаций с залоговым обеспечением серий А-01-FLB, А-01-PRT_BST_RUR-004, А-01-PRT_RZD_EUR-003 и А-01-PRT_RZD_GBP-005, предназначенных для квалифицированных инвесторов, размещенных в рамках программы облигаций серии A-01, общества с ограниченной ответственностью «Специализированное финансовое общество Финансовые инструменты 1» (г. Москва), регистрационные номера выпусков 6-01-00669-R-001P, 6-03-00669-R-001P, 6-04-00669-R-001P и 6-05-00669-R-001P.<|end|>
<|assistant|>
6-01-00669-R-001P, 6-03-00669-R-0

In [40]:
mlflow.set_experiment("Phi-3.5-mini-instruct fine-tuning")

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    eval_dataset = dataset['test'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        #max_steps = 120,
        num_train_epochs = 3,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 1994,
        evaluation_strategy = "steps",
        output_dir = "outputs",
        run_name=f"Phi-3.5-mini-instruct-{datetime.now().strftime('%Y-%m-%d-%H-%M-%s')}",
        report_to = "mlflow",
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/120 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/30 [00:00<?, ? examples/s]

In [41]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
3.01 GB of memory reserved.


In [42]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 120 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 45
 "-____-"     Number of trainable parameters = 29,884,416


Step,Training Loss,Validation Loss
1,0.414100,0.354525
2,0.476800,0.352378
3,0.469500,0.351218
4,0.376600,0.346027
5,0.590500,0.338172
6,0.263300,0.334425
7,0.289300,0.326802
8,0.235700,0.318706
9,0.175600,0.313398
10,0.316300,0.305156


In [43]:
!zip -r mlruns.zip /content/mlruns

  adding: content/mlruns/ (stored 0%)
  adding: content/mlruns/.trash/ (stored 0%)
  adding: content/mlruns/724041549191429121/ (stored 0%)
  adding: content/mlruns/724041549191429121/c73bb4adf2054650ab58dce8cb444b7f/ (stored 0%)
  adding: content/mlruns/724041549191429121/c73bb4adf2054650ab58dce8cb444b7f/tags/ (stored 0%)
  adding: content/mlruns/724041549191429121/c73bb4adf2054650ab58dce8cb444b7f/tags/mlflow.user (stored 0%)
  adding: content/mlruns/724041549191429121/c73bb4adf2054650ab58dce8cb444b7f/tags/mlflow.source.type (stored 0%)
  adding: content/mlruns/724041549191429121/c73bb4adf2054650ab58dce8cb444b7f/tags/mlflow.runName (deflated 4%)
  adding: content/mlruns/724041549191429121/c73bb4adf2054650ab58dce8cb444b7f/tags/mlflow.source.name (deflated 5%)
  adding: content/mlruns/724041549191429121/c73bb4adf2054650ab58dce8cb444b7f/params/ (stored 0%)
  adding: content/mlruns/724041549191429121/c73bb4adf2054650ab58dce8cb444b7f/params/prediction_loss_only (stored 0%)
  adding: conten

In [44]:
from google.colab import files
files.download("mlruns.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [45]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

828.5978 seconds used for training.
13.81 minutes used for training.
Peak reserved memory = 13.166 GB.
Peak reserved memory for training = 10.156 GB.
Peak reserved memory % of max memory = 89.273 %.
Peak reserved memory for training % of max memory = 68.864 %.


In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer, ) # token = "hf_..."
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, ) # token = "hf_..."

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", ) # token = "hf_..."

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "") # token = "hf_..."


In [ ]:
# Save to multiple GGUF options - much faster if you want multiple!
if True:
    model.push_to_hub_gguf(
        "winterForestStump/Phi-3.5-instruct-CBR_Bonds_info", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = '' # Get a token at https://huggingface.co/settings/tokens
    )

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.3G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 3.6 out of 12.67 RAM for saving.


100%|██████████| 32/32 [00:02<00:00, 13.03it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving winterForestStump/Phi-3.5-instruct-CBR_Bonds_info/pytorch_model-00001-of-00002.bin...
Unsloth: Saving winterForestStump/Phi-3.5-instruct-CBR_Bonds_info/pytorch_model-00002-of-00002.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = True.


...

unsloth.Q5_K_M.gguf:   0%|          | 0.00/2.72G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/winterForestStump/Phi-3.5-instruct-CBR_Bonds_info


No files have been modified since last commit. Skipping to prevent empty commit.


Saved Ollama Modelfile to https://huggingface.co/winterForestStump/Phi-3.5-instruct-CBR_Bonds_info
